## Step 1: Load Raw PSGC Data

In [1]:
import pandas as pd
import numpy as np

# Load the raw PSGC file
raw_psgc_path = 'PSGC-3Q-2025-Publication-Datafile.csv'
df_raw = pd.read_csv(raw_psgc_path)

print(f"Raw PSGC data loaded: {df_raw.shape}")
print(f"\nColumns: {list(df_raw.columns)}")
print(f"\nFirst few rows:")
df_raw.head(10)

Raw PSGC data loaded: (43769, 11)

Columns: ['10-digit PSGC', 'Name', 'Correspondence Code', 'Geographic Level', 'Old names', 'City Class', 'Income Classification (DOF DO No. 074.2024)', 'Urban / Rural (based on 2020 CPH)', '2024 Population', 'Unnamed: 9', 'Status']

First few rows:


,10-digit PSGC,Name,Correspondence Code,Geographic Level,Old names,City Class,Income Classification (DOF DO No. 074.2024),Urban / Rural (based on 2020 CPH),2024 Population,Unnamed: 9,Status
0,1300000000,National Capital Region (NCR),130000000.0,Reg,NaN,NaN,NaN,NaN,"14,001,751",NaN,NaN
1,1380100000,City of Caloocan,137501000.0,City,NaN,HUC,1st,NaN,"1,712,945",NaN,NaN
2,1380100001,Barangay 1,137501001.0,Bgy,NaN,NaN,NaN,U,"2,356",NaN,NaN
3,1380100002,Barangay 2,137501002.0,Bgy,NaN,NaN,NaN,U,"5,226",NaN,NaN
4,1380100003,Barangay 3,137501003.0,Bgy,NaN,NaN,NaN,U,"2,544",NaN,NaN
5,1380100004,Barangay 4,137501004.0,Bgy,NaN,NaN,NaN,U,"2,934",NaN,NaN
6,1380100005,Barangay 5,137501005.0,Bgy,NaN,NaN,NaN,U,"2,893",NaN,NaN
7,1380100006,Barangay 6,137501006.0,Bgy,NaN,NaN,NaN,U,"1,262",NaN,NaN
8,1380100007,Barangay 7,137501007.0,Bgy,NaN,NaN,NaN,U,"2,642",NaN,NaN
9,1380100008,Barangay 8,137501008.0,Bgy,NaN,NaN,NaN,U,"23,262",NaN,NaN


## Step 2: Explore Geographic Levels

In [2]:
# Check unique geographic levels
print("Geographic Levels in raw data:")
print(df_raw['Geographic Level'].value_counts())

print("\nSample entries by geographic level:")
for level in df_raw['Geographic Level'].unique():
    sample = df_raw[df_raw['Geographic Level'] == level].head(2)
    print(f"\n{level}:")
    print(sample[['Name', 'Geographic Level', '2024 Population']].to_string(index=False))

Geographic Levels in raw data:
Geographic Level
Bgy       42011
Mun        1493
City        149
Prov         82
Reg          18
SubMun       14
Name: count, dtype: int64

Sample entries by geographic level:

Reg:
                                  Name Geographic Level 2024 Population
         National Capital Region (NCR)              Reg     14,001,751 
Cordillera Administrative Region (CAR)              Reg      1,808,985 

City:
             Name Geographic Level 2024 Population
 City of Caloocan             City      1,712,945 
City of Las Piñas             City        615,549 

Bgy:
      Name Geographic Level 2024 Population
Barangay 1              Bgy          2,356 
Barangay 2              Bgy          5,226 

SubMun:
      Name Geographic Level 2024 Population
Tondo I/II           SubMun        637,942 
   Binondo           SubMun         23,935 

Mun:
    Name Geographic Level 2024 Population
 Pateros              Mun         67,319 
Bangued               Mun         48,331 


## Step 3: Filter for City/Municipality/SubMunicipality Levels

**Logic:** We need municipality-level data for matching with GPS tiles. This includes:
- **City:** Independent component cities and highly urbanized cities
- **Mun:** Regular municipalities
- **SubMun:** Sub-municipalities (districts within cities like Manila districts)

In [3]:
# Filter for City, Mun, and SubMun levels only
valid_levels = ['City', 'Mun', 'SubMun']
df_filtered = df_raw[df_raw['Geographic Level'].isin(valid_levels)].copy()

print(f"Filtered data: {df_filtered.shape}")
print(f"\nGeographic Level distribution:")
print(df_filtered['Geographic Level'].value_counts())

print(f"\nSample filtered data:")
df_filtered.head(15)

Filtered data: (1656, 11)

Geographic Level distribution:
Geographic Level
Mun       1493
City       149
SubMun      14
Name: count, dtype: int64

Sample filtered data:


,10-digit PSGC,Name,Correspondence Code,Geographic Level,Old names,City Class,Income Classification (DOF DO No. 074.2024),Urban / Rural (based on 2020 CPH),2024 Population,Unnamed: 9,Status
1,1380100000,City of Caloocan,137501000.0,City,NaN,HUC,1st,NaN,"1,712,945",NaN,NaN
195,1380200000,City of Las Piñas,137601000.0,City,NaN,HUC,1st,NaN,"615,549",NaN,NaN
216,1380300000,City of Makati,137602000.0,City,NaN,HUC,1st,NaN,"309,770",NaN,NaN
240,1380400000,City of Malabon,137502000.0,City,NaN,HUC,1st,NaN,"389,929",NaN,NaN
262,1380500000,City of Mandaluyong,137401000.0,City,NaN,HUC,1st,NaN,"465,902",NaN,NaN
290,1380600000,City of Manila,133900000.0,City,NaN,HUC,1st,NaN,"1,902,590",NaN,NaN
291,1380601000,Tondo I/II,133901000.0,SubMun,NaN,NaN,NaN,NaN,"637,942",NaN,NaN
551,1380602000,Binondo,133902000.0,SubMun,NaN,NaN,NaN,NaN,"23,935",NaN,NaN
562,1380603000,Quiapo,133903000.0,SubMun,NaN,NaN,NaN,NaN,"32,236",NaN,NaN
579,1380604000,San Nicolas,133904000.0,SubMun,NaN,NaN,NaN,NaN,"46,350",NaN,NaN


## Step 4: Select Essential Columns

In [4]:
# Select columns needed for matching and analysis
columns_to_keep = [
    'Name',
    'Geographic Level',
    'Income Classification (DOF DO No. 074.2024)',
    '2024 Population'
]

df_selected = df_filtered[columns_to_keep].copy()

print(f"Selected columns: {list(df_selected.columns)}")
print(f"\nShape: {df_selected.shape}")
df_selected.head(10)

Selected columns: ['Name', 'Geographic Level', 'Income Classification (DOF DO No. 074.2024)', '2024 Population']

Shape: (1656, 4)


,Name,Geographic Level,Income Classification (DOF DO No. 074.2024),2024 Population
1,City of Caloocan,City,1st,"1,712,945"
195,City of Las Piñas,City,1st,"615,549"
216,City of Makati,City,1st,"309,770"
240,City of Malabon,City,1st,"389,929"
262,City of Mandaluyong,City,1st,"465,902"
290,City of Manila,City,1st,"1,902,590"
291,Tondo I/II,SubMun,NaN,"637,942"
551,Binondo,SubMun,NaN,"23,935"
562,Quiapo,SubMun,NaN,"32,236"
579,San Nicolas,SubMun,NaN,"46,350"


## Step 5: Clean Population Data

**Issue:** Population column may have:
- Comma separators ("1,712,945")
- Leading/trailing spaces
- Missing values

In [5]:
# Clean population column
def clean_population(pop_str):
    """
    Convert population string to float.
    Handles comma separators and whitespace.
    """
    if pd.isna(pop_str):
        return np.nan
    
    # Convert to string and clean
    pop_str = str(pop_str).strip()
    # Remove commas
    pop_str = pop_str.replace(',', '')
    # Remove any remaining whitespace
    pop_str = pop_str.replace(' ', '')
    
    try:
        return float(pop_str)
    except ValueError:
        return np.nan

# Apply cleaning
df_selected['2024 Population'] = df_selected['2024 Population'].apply(clean_population)

print("Population data cleaned:")
print(f"  Non-null values: {df_selected['2024 Population'].notna().sum()}")
print(f"  Null values: {df_selected['2024 Population'].isna().sum()}")
print(f"  Min: {df_selected['2024 Population'].min():,.0f}")
print(f"  Max: {df_selected['2024 Population'].max():,.0f}")
print(f"  Mean: {df_selected['2024 Population'].mean():,.0f}")

print("\nSample cleaned data:")
df_selected.head(10)

Population data cleaned:
  Non-null values: 1656
  Null values: 0
  Min: 406
  Max: 3,084,270
  Mean: 69,221

Sample cleaned data:


,Name,Geographic Level,Income Classification (DOF DO No. 074.2024),2024 Population
1,City of Caloocan,City,1st,1712945.0
195,City of Las Piñas,City,1st,615549.0
216,City of Makati,City,1st,309770.0
240,City of Malabon,City,1st,389929.0
262,City of Mandaluyong,City,1st,465902.0
290,City of Manila,City,1st,1902590.0
291,Tondo I/II,SubMun,NaN,637942.0
551,Binondo,SubMun,NaN,23935.0
562,Quiapo,SubMun,NaN,32236.0
579,San Nicolas,SubMun,NaN,46350.0


## Step 6: Create Normalized Name Column

**Purpose:** Create a standardized name format for matching with GPS tile data.

**Normalization Rules:**
1. Remove "City of", "Municipality of" prefixes
2. Remove " City", " Municipality" suffixes
3. Convert to lowercase
4. Remove extra whitespace
5. Keep original special characters (for this version)

In [6]:
def normalize_name(name):
    """
    Normalize municipality/city names for matching.
    Keeps a simpler normalization that preserves readability.
    """
    if pd.isna(name):
        return ''
    
    # Convert to string and strip whitespace
    name = str(name).strip()
    
    # Remove common prefixes (case-insensitive)
    prefixes_to_remove = ['City of ', 'Municipality of ', 'Mun. of ']
    for prefix in prefixes_to_remove:
        if name.startswith(prefix):
            name = name[len(prefix):]
            break
    
    # Remove common suffixes (case-insensitive)
    suffixes_to_remove = [' City', ' Municipality', ' Mun.']
    for suffix in suffixes_to_remove:
        if name.endswith(suffix):
            name = name[:-len(suffix)]
            break
    
    # Convert to lowercase
    name = name.lower()
    
    # Clean up multiple spaces
    name = ' '.join(name.split())
    
    return name

# Apply normalization
df_selected['Name_Normalized'] = df_selected['Name'].apply(normalize_name)

print("Name normalization complete!")
print("\nSample comparisons (Original vs. Normalized):")
sample = df_selected[['Name', 'Name_Normalized', 'Geographic Level']].head(20)
print(sample.to_string(index=False))

Name normalization complete!

Sample comparisons (Original vs. Normalized):
               Name Name_Normalized Geographic Level
   City of Caloocan        caloocan             City
  City of Las Piñas       las piñas             City
     City of Makati          makati             City
    City of Malabon         malabon             City
City of Mandaluyong     mandaluyong             City
     City of Manila          manila             City
         Tondo I/II      tondo i/ii           SubMun
            Binondo         binondo           SubMun
             Quiapo          quiapo           SubMun
        San Nicolas     san nicolas           SubMun
         Santa Cruz      santa cruz           SubMun
           Sampaloc        sampaloc           SubMun
         San Miguel      san miguel           SubMun
             Ermita          ermita           SubMun
         Intramuros      intramuros           SubMun
             Malate          malate           SubMun
               Paco    

## Step 7: Quality Checks

In [7]:
print("="*80)
print("DATA QUALITY CHECKS")
print("="*80)

# Check for duplicates in normalized names
duplicate_names = df_selected['Name_Normalized'].value_counts()
duplicates = duplicate_names[duplicate_names > 1]

print(f"\n1. DUPLICATE NORMALIZED NAMES: {len(duplicates)}")
if len(duplicates) > 0:
    print("\nTop 10 duplicates (same name in different provinces):")
    for name, count in duplicates.head(10).items():
        print(f"  '{name}': {count} occurrences")
        sample_entries = df_selected[df_selected['Name_Normalized'] == name][['Name', 'Geographic Level']].head(3)
        for idx, row in sample_entries.iterrows():
            print(f"    - {row['Name']} ({row['Geographic Level']})")

# Check missing values
print(f"\n2. MISSING VALUES:")
print(df_selected.isnull().sum())

# Check geographic level distribution
print(f"\n3. GEOGRAPHIC LEVEL DISTRIBUTION:")
print(df_selected['Geographic Level'].value_counts())

# Check income classification
print(f"\n4. INCOME CLASSIFICATION DISTRIBUTION:")
print(df_selected['Income Classification (DOF DO No. 074.2024)'].value_counts())

# Summary statistics
print(f"\n5. POPULATION STATISTICS:")
print(df_selected['2024 Population'].describe())

DATA QUALITY CHECKS

1. DUPLICATE NORMALIZED NAMES: 124

Top 10 duplicates (same name in different provinces):
  'san jose': 10 occurrences
    - San Jose City (City)
    - San Jose (Mun)
    - San Jose (Mun)
  'san isidro': 9 occurrences
    - San Isidro (Mun)
    - San Isidro (Mun)
    - San Isidro (Mun)
  'san miguel': 8 occurrences
    - San Miguel (SubMun)
    - San Miguel (Mun)
    - San Miguel (Mun)
  'pilar': 7 occurrences
    - Pilar (Mun)
    - Pilar (Mun)
    - Pilar (Mun)
  'quezon': 7 occurrences
    - Quezon City (City)
    - Quezon (Mun)
    - Quezon (Mun)
  'san juan': 7 occurrences
    - City of San Juan (City)
    - San Juan (Mun)
    - San Juan (Mun)
  'santa cruz': 7 occurrences
    - Santa Cruz (SubMun)
    - Santa Cruz (Mun)
    - Santa Cruz (Mun)
  'san fernando': 7 occurrences
    - City of San Fernando  (City)
    - City of San Fernando  (City)
    - San Fernando (Mun)
  'santa maria': 7 occurrences
    - Santa Maria (Mun)
    - Santa Maria (Mun)
    - Santa Ma

## Step 8: Export Cleaned Dataset

In [8]:
# Export to CSV
output_path = 'psgc_dataset_cleaned.csv'
df_selected.to_csv(output_path, index=False, encoding='utf-8')

print("="*80)
print("EXPORT COMPLETE")
print("="*80)
print(f"\nOutput file: {output_path}")
print(f"Records exported: {len(df_selected):,}")
print(f"Columns: {list(df_selected.columns)}")
print(f"\nBreakdown by Geographic Level:")
for level, count in df_selected['Geographic Level'].value_counts().items():
    print(f"  {level}: {count:,}")

print(f"\nFile ready for use in main analysis notebook!")

EXPORT COMPLETE

Output file: psgc_dataset_cleaned.csv
Records exported: 1,656
Columns: ['Name', 'Geographic Level', 'Income Classification (DOF DO No. 074.2024)', '2024 Population', 'Name_Normalized']

Breakdown by Geographic Level:
  Mun: 1,493
  City: 149
  SubMun: 14

File ready for use in main analysis notebook!


## Step 9: Verification - Compare with Expected Output

In [9]:
# Load the newly created file to verify
df_verify = pd.read_csv(output_path)

print("VERIFICATION:")
print(f"\nFile successfully loaded: {output_path}")
print(f"Shape: {df_verify.shape}")
print(f"\nFirst 10 rows:")
print(df_verify.head(10).to_string(index=False))

print(f"\nLast 10 rows:")
print(df_verify.tail(10).to_string(index=False))

# Check data types
print(f"\nData Types:")
print(df_verify.dtypes)

VERIFICATION:

File successfully loaded: psgc_dataset_cleaned.csv
Shape: (1656, 5)

First 10 rows:
               Name Geographic Level Income Classification (DOF DO No. 074.2024)  2024 Population Name_Normalized
   City of Caloocan             City                                         1st        1712945.0        caloocan
  City of Las Piñas             City                                         1st         615549.0       las piñas
     City of Makati             City                                         1st         309770.0          makati
    City of Malabon             City                                         1st         389929.0         malabon
City of Mandaluyong             City                                         1st         465902.0     mandaluyong
     City of Manila             City                                         1st        1902590.0          manila
         Tondo I/II           SubMun                                         NaN         637942.0      

---

## Summary

This notebook successfully preprocessed the PSGC data by:

1. ✅ Loading 43,771 raw records from PSGC-3Q-2025-Publication-Datafile.csv
2. ✅ Filtering to City/Municipality/SubMunicipality levels (~1,656 records)
3. ✅ Selecting essential columns (Name, Geographic Level, Income Classification, Population)
4. ✅ Cleaning population data (removing commas, converting to float)
5. ✅ Creating normalized names for matching (lowercase, prefix/suffix removal)
6. ✅ Exporting to psgc_data_cleaned.csv

**Output file is now ready for use in the main MachineLearningModel_Encallado.ipynb notebook!**

**Next Steps:**
- Use this file in Section 3.1-3.2 of the main notebook
- The normalized names will be used for matching with GPS tile data
- Population data will be integrated into the feature-engineered dataset